# Group And Resample
> Aggregate on selected columns

In [ ]:
#| default_exp etl.national.group_and_resample

In [ ]:
#| export

from typing import Dict, Union
import pandas as pd

In [ ]:
#| export
PANDAS_FREQS = {'W-SUN': 'Weekly frequency (Sundays)',
 'W-MON': 'Weekly frequency (Mondays)',
 'W-TUE': 'Weekly frequency (Tuesdays)',
 'W-WED': 'Weekly frequency (Wednesdays)',
 'W-THU': 'Weekly frequency (Thursdays)',
 'W-FRI': 'Weekly frequency (Fridays)',
 'W-SAT': 'Weekly frequency (Saturdays)'}

In [ ]:
#| export

import os
import pathlib
import pandas as pd
import unidecode

from niloIQ2.etl import tables

old_version = '03'
freq = 'W-MON'
new_version = '04_'+freq

table: tables.Table

for table in tables.national_list:

    df = pd.read_csv(
        table.version(old_version),
        low_memory=False,
    )

    df['date'] = pd.to_datetime(df['date'], )
    
    
    df.set_index(['state', 'product', 'date'], inplace=True)
    df.sort_values(['state', 'product', 'date'], inplace=True)

    level_values =  [df.index.get_level_values(i) for i in [0, 1]] 

    result = df.groupby(
       level_values + [pd.Grouper(freq=freq, level=-1)]
    ).agg(
        {
            'min_price': min,
            'max_price': max,
            'price': 'mean', 
            'import': 'first'
        }
    )
    
    result.to_csv(table.version(new_version))
    